In [24]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver, InMemorySaver
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import Literal, Annotated, TypedDict
from pydantic import BaseModel, Field
from dotenv import load_dotenv
load_dotenv()
import time
llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [4]:
class jokestart(TypedDict):
    topic: str
    joke: str 
    explaination: str

In [5]:
def generate_joke(state: jokestart):
    prompt= f"Generate a joke on topic - {state['topic']}"

    respone= llm.invoke(prompt).content 
    return {'joke':respone}

In [6]:
def generate_explaination(state:jokestart):
    prompt= f"Generate a explaination on joke - {state['joke']}"
    
    response= llm.invoke(prompt).content 
    return {'explaination':response}


In [11]:
graph= StateGraph(jokestart)

graph.add_node('generate_joke',generate_joke)
graph.add_node('generate_explaination', generate_explaination)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke','generate_explaination')
graph.add_edge('generate_explaination',END)

checkpointer= MemorySaver()

workflow= graph.compile(checkpointer=checkpointer)

In [12]:
config1= {'configurable':{'thread_id':'1'}}
workflow.invoke({'topic':'pizza'},config=config1)

{'topic': 'pizza',
 'joke': "Why does the pizza slice like the mushroom so much?\n\nBecause he's a fungi!",
 'explaination': 'This joke relies on a pun, using the similar-sounding words "fungi" and "fun guy."\n\n* **Fungi:**  This is the plural of fungus, a type of organism like mushrooms.\n\n* **Fun guy:** This is a playful phrase describing someone who is enjoyable to be around.\n\nThe joke sets up the expectation that the pizza slice might like the mushroom for some culinary reason.  Instead, the punchline unexpectedly uses "fungi" to create a humorous connection to "fun guy," implying the pizza slice enjoys the mushroom\'s company because the mushroom is a fun companion.  The unexpected shift in meaning is what makes the joke funny.'}

In [13]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': "Why does the pizza slice like the mushroom so much?\n\nBecause he's a fungi!", 'explaination': 'This joke relies on a pun, using the similar-sounding words "fungi" and "fun guy."\n\n* **Fungi:**  This is the plural of fungus, a type of organism like mushrooms.\n\n* **Fun guy:** This is a playful phrase describing someone who is enjoyable to be around.\n\nThe joke sets up the expectation that the pizza slice might like the mushroom for some culinary reason.  Instead, the punchline unexpectedly uses "fungi" to create a humorous connection to "fun guy," implying the pizza slice enjoys the mushroom\'s company because the mushroom is a fun companion.  The unexpected shift in meaning is what makes the joke funny.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06ead1-c3a6-697f-8002-b59025534c99'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}, 'thread_id': '1'}, created_at='2025-08-01T07:

In [14]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': "Why does the pizza slice like the mushroom so much?\n\nBecause he's a fungi!", 'explaination': 'This joke relies on a pun, using the similar-sounding words "fungi" and "fun guy."\n\n* **Fungi:**  This is the plural of fungus, a type of organism like mushrooms.\n\n* **Fun guy:** This is a playful phrase describing someone who is enjoyable to be around.\n\nThe joke sets up the expectation that the pizza slice might like the mushroom for some culinary reason.  Instead, the punchline unexpectedly uses "fungi" to create a humorous connection to "fun guy," implying the pizza slice enjoys the mushroom\'s company because the mushroom is a fun companion.  The unexpected shift in meaning is what makes the joke funny.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06ead1-c3a6-697f-8002-b59025534c99'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}, 'thread_id': '1'}, created_at='2025-08-01T07

In [15]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': "Why did the Italian chef quit his job?  Because he didn't get enough *pasta*bilities!",
 'explaination': 'This joke relies on a pun, using the similar-sounding words "pasta" and "possibilities."\n\nThe setup creates an expectation of a typical reason for quitting a job, perhaps low pay or difficult working conditions.  Instead, the punchline uses "pasta-bilities" – a playful combination of "pasta" (a key ingredient in Italian cuisine) and "possibilities" – to imply the chef felt his job lacked opportunities or creative freedom.  The humor comes from the unexpected and slightly absurd connection between the culinary world and the concept of career prospects.  It\'s funny because it\'s a clever wordplay that subverts the initial serious tone of the question.'}

In [16]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': "Why does the pizza slice like the mushroom so much?\n\nBecause he's a fungi!", 'explaination': 'This joke relies on a pun, using the similar-sounding words "fungi" and "fun guy."\n\n* **Fungi:**  This is the plural of fungus, a type of organism like mushrooms.\n\n* **Fun guy:** This is a playful phrase describing someone who is enjoyable to be around.\n\nThe joke sets up the expectation that the pizza slice might like the mushroom for some culinary reason.  Instead, the punchline unexpectedly uses "fungi" to create a humorous connection to "fun guy," implying the pizza slice enjoys the mushroom\'s company because the mushroom is a fun companion.  The unexpected shift in meaning is what makes the joke funny.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06ead1-c3a6-697f-8002-b59025534c99'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}, 'thread_id': '1'}, created_at='2025-08-01T07:

# Time Travel

In [17]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06ead1-b676-6e30-8001-4bb5a24c448d"}})

StateSnapshot(values={'topic': 'pizza', 'joke': "Why does the pizza slice like the mushroom so much?\n\nBecause he's a fungi!"}, next=('generate_explaination',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f06ead1-b676-6e30-8001-4bb5a24c448d'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}, 'thread_id': '1'}, created_at='2025-08-01T07:57:01.664619+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06ead1-ae8c-68fd-8000-009a895d9dc0'}}, tasks=(PregelTask(id='a5ba29e5-114e-4941-ccc3-82aafac0d01d', name='generate_explaination', path=('__pregel_pull', 'generate_explaination'), error=None, interrupts=(), state=None, result={'explaination': 'This joke relies on a pun, using the similar-sounding words "fungi" and "fun guy."\n\n* **Fungi:**  This is the plural of fungus, a type of organism like mushrooms.\n\n* **Fun guy:** This is a playful phrase describing someone who is enjoyable to be around.\n\nThe joke sets up the 

In [18]:
workflow.invoke(None, {'configurable':{'thread_id':'1', 'checkpoint_id':'1f06ead1-b676-6e30-8001-4bb5a24c448d'}})

{'topic': 'pizza',
 'joke': "Why does the pizza slice like the mushroom so much?\n\nBecause he's a fungi!",
 'explaination': 'This joke relies on a pun, using the similar-sounding words "fungi" and "fun guy."\n\n* **Fungi:**  Refers to a kingdom of organisms including mushrooms, molds, and yeasts.\n* **Fun guy:**  Means a person who is enjoyable to be around, someone who is fun.\n\nThe joke sets up the expectation that the pizza slice liking the mushroom will have a straightforward, possibly culinary, explanation.  Instead, it uses the word "fungi" to create a humorous double meaning, implying the pizza slice enjoys the mushroom\'s company because the mushroom is a "fun guy."  The unexpected shift from a literal to a playful interpretation is what makes the joke funny.'}

In [19]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': "Why does the pizza slice like the mushroom so much?\n\nBecause he's a fungi!", 'explaination': 'This joke relies on a pun, using the similar-sounding words "fungi" and "fun guy."\n\n* **Fungi:**  Refers to a kingdom of organisms including mushrooms, molds, and yeasts.\n* **Fun guy:**  Means a person who is enjoyable to be around, someone who is fun.\n\nThe joke sets up the expectation that the pizza slice liking the mushroom will have a straightforward, possibly culinary, explanation.  Instead, it uses the word "fungi" to create a humorous double meaning, implying the pizza slice enjoys the mushroom\'s company because the mushroom is a "fun guy."  The unexpected shift from a literal to a playful interpretation is what makes the joke funny.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06ead7-fd3e-6e1d-8002-d686eae6322b'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}, 'thread_id'

# Fault Tolerance

In [21]:
class CrashState(TypedDict):
    input: str
    step1: str 
    step2: str 

In [22]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1000)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [25]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [26]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
❌ Kernel manually interrupted (crash simulated).


In [ ]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)